<a href="https://colab.research.google.com/github/gulabpatel/FastAPI/blob/main/02%3A%20Deploy_ML_Models_as_Service_with_FastAPI_%26_ColabCode_from_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video walkthrough the code : https://www.youtube.com/watch?v=7-igAakUUTY

In [ ]:
!pip install colabcode
!pip install fastapi

In [2]:
from colabcode import ColabCode
from fastapi import FastAPI

In [3]:
cc = ColabCode(port=12000, code=False)

In [6]:
app = FastAPI()

@app.get("/")
async def read_root():
  return {"message": "Welcome to the Future!"}

In [7]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://fd4d00e7a46a.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [166]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [166]


In [8]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

iris = load_iris()
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
model_f = model.fit(X_train, y_train)

print("Model score: ", model.score(X_train, y_train))
print("Test Accuracy: ", model.score(X_test, y_test))

pickle.dump(model_f, open("model_gb.pkl", "wb"))

Model score:  0.9629629629629629
Test Accuracy:  0.8666666666666667


In [9]:
%%writefile models.py
from pydantic import BaseModel, conlist
from typing import List


class Iris(BaseModel):
    data: List[conlist(float, min_items=4, max_items=4)]

Writing models.py


In [10]:
import pickle
import logging
from fastapi import FastAPI
from models import Iris

app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# # Initialize logging
# my_logger = logging.getLogger()
# my_logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_gb.pkl", "rb"))

@app.post("/api", tags=["prediction"])
async def get_predictions(iris: Iris):
    try:
        data = dict(iris)['data']
        print(data)
        iris_types = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        prediction = list(map(lambda x: iris_types[x], model.predict(data).tolist()))
        log_proba = model.predict_log_proba(data).tolist()
        return {"prediction": prediction, "log_proba": log_proba}
    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://ba9d1e0799d4.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [166]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "GET /openapi.json HTTP/1.1" 200 OK
[[4.5, 3.5, 2.5, 1.5]]
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "POST /api HTTP/1.1" 200 OK
[[3.0, 2.0, 1.0, 3.5]]
INFO:     2409:4063:6e13:69ad:e026:1ab5:ba8f:c7ec:0 - "POST /api HTTP/1.1" 200 OK


##### To run on browser click on : https://ba9d1e0799d4.ngrok.io/docs